`Steps`

1 .  Explore and Clean the Data

2 . Split the data into training , validation and testing

3 . Fit the initial model and evaluate

4 .  Tune hyperparameters with RandomizedSearchCV

4 . 1 Use best parameters from RandomizedSearchCV to GridSearchCV

5 . Evaluate on a validation set

6 . Finalize the model selection and evaluate on the test set

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib
import warnings
# Filter out warnings
warnings.filterwarnings("ignore")



In [2]:
df = pd.read_csv("train.csv")
df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


# **Data Cleaning**


# Cleaning Numeric Column 

In [3]:
# Ensure 'float64' is in lowercase and fix inplace usage
df_numeric = df.select_dtypes(include=["int64", "float64"])

# Fill missing values in numeric columns with the median
df_numeric = df_numeric.fillna(df_numeric.median())

# If you want to apply the changes back to your original DataFrame:
df.update(df_numeric)


# **Clean object Column by turning it into category and used .codes to turn it into numbers**

In [4]:
df_object = df.select_dtypes(include='object')

# Convert object columns to categorical, then into numeric codes
for key in df_object.columns:
    df[key] = pd.Categorical(df_object[key]).codes+1 # Adding 1 to avoid -1 for 'Unknown'

# Update the original dataframe with the encoded values 
df.update(df)
    

In [5]:
if df.isna().sum().any() > 0:
    print("There is still null values in the Dataset")
else:
    print("No null Values in the dataset")

No null Values in the dataset


# **Split the data into training , validation and testing**

In [6]:
# splitting data into x and y 
x = df.drop("SalePrice" , axis=1)
y = df.SalePrice


In [7]:
#splitting x and y in training and test set
x_train, x_test ,y_train , y_test = train_test_split(x , y, 
                                                      test_size= 0.4,
                                                      random_state = 42)

# splitting test set into test and validation
x_val, x_test ,y_val , y_test = train_test_split(x_test , y_test, 
                                                      test_size= 0.5,
                                                      random_state = 42)

In [8]:
x_train.dtypes.value_counts()

int8       43
int64      34
float64     3
Name: count, dtype: int64

# **Fit the initial model and evaluate**

In [9]:
#Fitting initial
from sklearn.ensemble import RandomForestRegressor

#Instantiating a model
rf_model = RandomForestRegressor(n_jobs = -1, 
                                 random_state= 42)

# Fitting the model
rf_model.fit(x_train , y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [10]:
# function with every regression metric for evaluatuion and model performance

from sklearn.metrics import mean_absolute_error , mean_squared_error , mean_squared_log_error , r2_score 

# Function that calculates root mean sqaured error
def rmse(y_train , y_preds):
    
    "calculates the root mean squared error "
    
    return np.sqrt(mean_squared_error(y_train, y_preds))


# Function that calculates root mean squared log error
def rmsle(y_train , y_preds):
    "calculates the root mean squared log error"
    
    return np.sqrt(mean_squared_log_error(y_train , y_preds))



# Function for all regression metrics
def all_metrics(model):
    y_train_preds = model.predict(x_train)
    y_val_preds = model.predict(x_val)
    
    scores = { 
             # Training Metrics
        "Training Metrics" : {
             "Trained MSE" : float(mean_absolute_error(y_train , y_train_preds)) ,
             "Trained MSE" : float(mean_squared_error(y_train , y_train_preds)) , 
             "Trained MSLE" : float(mean_squared_log_error(y_train , y_train_preds)) , 
             "Trained R^2" : round(float(r2_score(y_train , y_train_preds) * 100) , 2),
             "Trained RMSE" : float(rmse(y_train , y_train_preds)),
             "Trained RMSLE" : float(rmsle(y_train , y_train_preds)),
             
        } ,
          
              
# The presence of np.float64 in the output is due to the fact that numpy uses this specific type for numerical values. 
# However, when displaying the output, it makes the type explicit, which you don’t necessarily need.
# To remove this and display the results as regular floating-point numbers, 
# you can convert the np.float64 values to Python native floats using the float() function. 

          
             # Validation metrics
        "Validation Metrics" : {     
             "Validation MSE" : float(mean_absolute_error(y_val , y_val_preds)) ,
             "Validation MSE" : float(mean_squared_error(y_val , y_val_preds)) , 
             "Validation MSLE" : float(mean_squared_log_error(y_val , y_val_preds)) , 
             "Validation R^2" : round(float(r2_score(y_val , y_val_preds) * 100) , 2),
             "Validation RMSE" : float(rmse(y_val , y_val_preds)),
             "Validation RMSLE" : float(rmsle(y_val , y_val_preds)),
            }
    }
    
    # Display the metrics clearly
    print("Training Metrics:")
    for key, value in scores["Training Metrics"].items():
        print(f"{key}: {value}")
    
    print("\nValidation Metrics:")
    for key, value in scores["Validation Metrics"].items():
        print(f"{key}: {value}")
    
    
    
    
    

In [11]:
all_metrics(rf_model)

Training Metrics:
Trained MSE: 137234767.6708845
Trained MSLE: 0.003935487436800935
Trained R^2: 97.58
Trained RMSE: 11714.724395856887
Trained RMSLE: 0.06273346345293662

Validation Metrics:
Validation MSE: 1279279168.5928566
Validation MSLE: 0.024737738943255455
Validation R^2: 86.42
Validation RMSE: 35767.01229614876
Validation RMSLE: 0.15728235420178405


# **Tune hyperparameters**

## Choose the estimator / algorithm for the problem

## Boosting : Explore Boosting algorithms in Python

Import GradientBoostingRegressor and AdaBoostRegressor from sklearn and explore the hyperparameters

Import Boosting Algorithm for Regression

In [12]:
from sklearn.ensemble import GradientBoostingRegressor , AdaBoostRegressor

# GradientBoostingRegressor().get_params()

In [13]:
# AdaBoostRegressor().get_params()

In [14]:
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
np.random.seed(42)

gb = GradientBoostingRegressor(subsample=0.35)

parameters = {
    "n_estimators" : np.arange(50, 2000, 10) ,
    "max_depth" : [None, 3,20, 3 ],
    "learning_rate" : [0.01, 0.1, 1 , 0.05]
}

cv = RandomizedSearchCV(gb , 
                        param_distributions=parameters , 
                        cv = 5,
                        n_iter = 100 ,
                        verbose = 1)

cv.fit(x_train , y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(subsample=0.35),
                   n_iter=100,
                   param_distributions={'learning_rate': [0.01, 0.1, 1, 0.05],
                                        'max_depth': array([ 3,  6,  9, 12, 15, 18]),
                                        'n_estimators': array([  50,   60,   70,   80,   90,  100,  110,  120,  130,  140,  150,
        160,  170,  180,  190,  200,  210,  220,  230,  240,  250,  260,
        270,  280,  290,  300,  310,  320,  330,  340,  350,  360,  370,
        380,  390,  400,  410,  420,  430...
       1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360,
       1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470,
       1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580,
       1590, 1600, 1610, 1620, 1630, 1640, 1650, 1660, 1670, 1680, 1690,
       1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800,
       1810, 1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890, 1900, 1910,
       1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990])},
                   verbose=1)

In [15]:
# create a model with the best hyperparamaters

def best_parameters():
    # Get the best parameters from the search
    best_grid = cv.best_estimator_ 
    
    
    #Fitting the model
    best_grid.fit(x_train,y_train)
    
    return best_grid

gb_model = best_parameters()

In [16]:
gb_model 

GradientBoostingRegressor(learning_rate=0.05, max_depth=np.int64(3),
                          n_estimators=np.int64(1540), subsample=0.35)

In [17]:
# save and load the model
from joblib import load , dump

dump(gb_model, filename='gradient.joblib')
gb_model = load("gradient.joblib")
gb_model

GradientBoostingRegressor(learning_rate=0.05, max_depth=np.int64(3),
                          n_estimators=np.int64(1540), subsample=0.35)

## Bagging : Implement a bagging model

In this section , we will fit and evaluate a simple Random Forest Model

In [18]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_jobs= -1 , random_state=42 )
parameters = {
    "n_estimators" : np.arange(50, 2000, 10),
    "max_depth" : np.arange(3, 20 ,5) ,
    "max_features" : [None, "sqrt", "log2", 1, 1.0,] ,
    'min_samples_split'  : [15 , 20],
    "min_weight_fraction_leaf" : np.arange(0.0, 0.5, 11) ,
    "oob_score" : [False] ,
    "warm_start": [True]
}

cv = RandomizedSearchCV(rf ,
                  param_distributions = parameters ,
                  n_iter= 100 ,
                  cv = 5 ,
                  verbose = 1)

cv.fit(x_train , y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=100,
                   param_distributions={'max_depth': array([ 3,  8, 13, 18]),
                                        'max_features': [None, 'sqrt', 'log2',
                                                         1, 1.0],
                                        'min_samples_split': [15, 20],
                                        'min_weight_fraction_leaf': array([0.]),
                                        'n_estimators': array([  50,   60,   70,   80,   90,  100,  110,  120,  130,  140,  150,
        160,  170,  180,  190,  200,  210,  220,  230,  24...
       1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470,
       1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580,
       1590, 1600, 1610, 1620, 1630, 1640, 1650, 1660, 1670, 1680, 1690,
       1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800,
       1810, 1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890, 1900, 1910,
       1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990]),
                                        'oob_score': [False],
                                        'warm_start': [True]},
                   verbose=1)

In [19]:
# create a model with the best hyperparamaters

def best_parameters():
    # Get the best parameters from the search
    best_grid = cv.best_params_ 
    
    # Instantiate the model and Unpack the best_parameters dictionary into the RandomForestRegressor constructor (** means unpacking the best parameters)
    model = RandomForestRegressor(**best_grid)
    
    #Fitting the model
    model.fit(x_train,y_train)
    
    return model

rf_model = best_parameters()

In [20]:
rf_model

RandomForestRegressor(max_depth=np.int64(13), min_samples_split=15,
                      min_weight_fraction_leaf=np.float64(0.0),
                      n_estimators=np.int64(810), warm_start=True)

In [21]:
# save and load the model
from joblib import load , dump

dump(rf_model , filename='randomforest.joblib')
rf_model = load("randomforest.joblib")
rf_model

RandomForestRegressor(max_depth=np.int64(13), min_samples_split=15,
                      min_weight_fraction_leaf=np.float64(0.0),
                      n_estimators=np.int64(810), warm_start=True)

## Stacking : Explore stacking algorithms in Python

Import `StackingClassifier` from `sklearn` and explore the hyperparameters

Import Stacking Algorithm For Regression

In [22]:
from sklearn.ensemble import StackingRegressor, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression

# Define the base models (estimators) for stacking
estimators = [
    ('gb', GradientBoostingRegressor(random_state=42)),  # Use GradientBoostingRegressor for regression
    ('rf', RandomForestRegressor(random_state=42))       # Use RandomForestRegressor for regression
]

sc = StackingRegressor(estimators = estimators , final_estimator=LinearRegression())

In [23]:
parameters = {
    'gb__n_estimators' : np.arange(50, 2000, 10),
    'rf__n_estimators' : np.arange(50, 2000, 10),
    'passthrough' : [True , False]
}

cv = RandomizedSearchCV(sc , 
                        param_distributions = parameters , 
                        n_iter=100,
                        cv = 5 ,
                        random_state=42)

cv.fit(x_train, y_train)

In [25]:
# create a model with the best hyperparamaters

def best_parameters():
    
    # Get the best parameters from the search
    best_params = cv.best_params_ 
    
    # Extract base model parameters from best_params
    gb_n_estimators = best_params.get('gb__n_estimators', 100)
    rf_n_estimators = best_params.get('rf__n_estimators', 100)
    passthrough = best_params.get('passthrough', False)
    
    
     # Reinstantiate StackingRegressor with the best parameters
    # Create and fit the final model with the best parameters
    model = StackingRegressor(
        estimators=[
            ('gb', GradientBoostingRegressor(n_estimators=gb_n_estimators, random_state=42)),
            ('rf', RandomForestRegressor(n_estimators=rf_n_estimators, random_state=42))
        ],
        final_estimator=LinearRegression(),
        passthrough=passthrough
    )
    
    
    
    
    #Fitting the model
    model.fit(x_train,y_train)
    
    return model

sk_model = best_parameters()

In [26]:
sk_model

StackingRegressor(estimators=[('gb',
                               GradientBoostingRegressor(n_estimators=200,
                                                         random_state=42)),
                              ('rf', RandomForestRegressor(random_state=42))],
                  final_estimator=LinearRegression())

In [27]:
# save and load the model
from joblib import load , dump

dump(sk_model , filename='stackingregressor.joblib')
sk_model = load("stackingregressor.joblib")
sk_model

StackingRegressor(estimators=[('gb',
                               GradientBoostingRegressor(n_estimators=200,
                                                         random_state=42)),
                              ('rf', RandomForestRegressor(random_state=42))],
                  final_estimator=LinearRegression())

# XGBoost Model Algorithm

**Implement XGBosst Algorithm**

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()

parameters = {
    "learning_rate" : [None , 1 , 0.1]
}

cv = RandomizedSearchCV(xgb_model , 
                        cv = 5 , 
                        n_iter = 100 ,
                        param_distributions= parameters,
                        verbose= 1)

cv.fit(x_train , y_train)

In [ ]:
# create a model with the best hyperparamaters

def best_parameters():
    # Get the best parameters from the search
    best_grid = cv.best_params_ 
    
    # Instantiate the model and Unpack the best_parameters dictionary into the RandomForestRegressor constructor (** means unpacking the best parameters)
    model = XGBRegressor(**best_grid)
    
    #Fitting the model
    model.fit(x_train,y_train)
    
    return model

xgb_model = best_parameters()

In [ ]:
# save and load the model
from joblib import load , dump

dump(xgb_model , filename='stackingregressor.joblib')
xgb_model = load("stackingregressor.joblib")
xgb_model

# **Evaluate on the Validation set**



In [28]:
# function with every regression metric for evaluatuion and model performance

from sklearn.metrics import mean_absolute_error , mean_squared_error , mean_squared_log_error , r2_score 

# Function that calculates root mean sqaured error
def rmse(y_train , y_preds):
    
    "calculates the root mean squared error "
    
    return np.sqrt(mean_squared_error(y_train, y_preds))


# Function that calculates root mean squared log error
def rmsle(y_train , y_preds):
    "calculates the root mean squared log error"
    
    return np.sqrt(mean_squared_log_error(y_train , y_preds))



# Function for all regression metrics
def all_metrics():
    y_val_preds_gb = gb_model.predict(x_val)
    y_val_preds_rf = rf_model.predict(x_val)
    y_val_preds_sk = sk_model.predict(x_val)

    scores = { 
             "Validation Metrics (GradientBoostingRegressor)" : {     
             "Validation MSE" : float(mean_absolute_error(y_val ,y_val_preds_gb)) , 
             "Validation MSLE" : float(mean_squared_log_error(y_val , y_val_preds_gb)) , 
             "Validation R^2" : round(float(r2_score(y_val , y_val_preds_gb) * 100) , 2),
             "Validation RMSE" : float(rmse(y_val , y_val_preds_gb)),
             "Validation RMSLE" : float(rmsle(y_val , y_val_preds_gb)),
            } ,
        
        
            "Validation Metrics (RandomForestRegressor)" : {     
             "Validation MSE" : float(mean_absolute_error(y_val , y_val_preds_rf)) ,
             "Validation MSLE" : float(mean_squared_log_error(y_val , y_val_preds_rf)) , 
             "Validation R^2" : round(float(r2_score(y_val , y_val_preds_rf) * 100) , 2),
             "Validation RMSE" : float(rmse(y_val , y_val_preds_rf)),
             "Validation RMSLE" : float(rmsle(y_val , y_val_preds_rf)),
            } ,
        
        "Validation Metrics(StackingRegressor)" : {     
             "Validation MSE" : float(mean_absolute_error(y_val , y_val_preds_sk)) ,
             "Validation MSLE" : float(mean_squared_log_error(y_val ,y_val_preds_sk)) , 
             "Validation R^2" : round(float(r2_score(y_val ,y_val_preds_sk) * 100) , 2),
             "Validation RMSE" : float(rmse(y_val , y_val_preds_sk)),
             "Validation RMSLE" : float(rmsle(y_val , y_val_preds_sk)),
            }
    }
    

    
    print("\nValidation Metrics(GradientBoostingRegressor):")
    for key, value in scores["Validation Metrics (GradientBoostingRegressor)"].items():
        print(f"{key}: {value}")
    
    print("\nValidation Metrics(RandomForestRegressor):")
    for key, value in scores["Validation Metrics (RandomForestRegressor)"].items():
        print(f"{key}: {value}")
        
        
    print("\nValidation Metrics(StackingRegressor):")
    for key, value in scores["Validation Metrics(StackingRegressor)"].items():
        print(f"{key}: {value}")
    
    
all_metrics()
   
    

# **Finalize the model selection and evaluate on the test set**

In [80]:
from sklearn.metrics import mean_absolute_error , mean_squared_error , mean_squared_log_error , r2_score 

#GradientBoostingRegressor Model
def gb_metrics():
    y_test_preds_gb = gb_model.predict(x_test)
   

    scores = { 
             "Validation MSE" : float(mean_absolute_error(y_test ,y_test_preds_gb)) , 
             "Validation MSLE" : float(mean_squared_log_error(y_test , y_test_preds_gb)) , 
             "Validation R^2" : round(float(r2_score(y_test , y_test_preds_gb) * 100) , 2),
             "Validation RMSE" : float(rmse(y_test , y_test_preds_gb)),
             "Validation RMSLE" : float(rmsle(y_test , y_test_preds_gb))
    }
    
    return scores
gb_metrics()

{'Validation MSE': 14384.395143626229,
 'Validation MSLE': 0.016906928242261234,
 'Validation R^2': 91.19,
 'Validation RMSE': 20758.01211366287,
 'Validation RMSLE': 0.1300266443551522}

In [54]:
#RandomForestModel

def rf_metrics():
    y_test_preds_rf = rf_model.predict(x_test)
    
    
    
    scores = {
             "Validation MSE" : float(mean_absolute_error(y_test , y_test_preds_rf)) ,
             "Validation MSLE" : float(mean_squared_log_error(y_test , y_test_preds_rf)) , 
             "Validation R^2" : round(float(r2_score(y_test , y_test_preds_rf) * 100) , 2),
             "Validation RMSE" : float(rmse(y_test , y_test_preds_rf)),
             "Validation RMSLE" : float(rmsle(y_test , y_test_preds_rf)),
            } 
    
    return scores
rf_metrics()

In [83]:
#StackingRegressor Model
def sk_metrics():
    y_test_preds_sk = sk_model.predict(x_test)
    

    scores = {     
             "Validation MSE" : float(mean_absolute_error(y_val , y_test_preds_sk)) ,
             "Validation MSLE" : float(mean_squared_log_error(y_val ,y_test_preds_sk)) , 
             "Validation R^2" : round(float(r2_score(y_val ,y_test_preds_sk) * 100) , 2),
             "Validation RMSE" : float(rmse(y_val , y_test_preds_sk)),
             "Validation RMSLE" : float(rmsle(y_val , y_test_preds_sk)) ,
            }

    return scores

sk_metrics()

{'Validation MSE': 84773.59360514957,
 'Validation MSLE': 0.34097671651851574,
 'Validation R^2': -55.07,
 'Validation RMSE': 120844.23682624308,
 'Validation RMSLE': 0.5839321163615817}

## Import test data

In [56]:
df_test = pd.read_csv("test.csv")
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [57]:
# Ensure 'float64' is in lowercase and fix inplace usage
df_numeric = df_test.select_dtypes(include=["int64", "float64"])

# Fill missing values in numeric columns with the median
df_numeric = df_numeric.fillna(df_numeric.median())

# If you want to apply the changes back to your original DataFrame:
df_test.update(df_numeric)


In [58]:
df_object = df_test.select_dtypes(include='object')

# Convert object columns to categorical, then into numeric codes
for key in df_object.columns:
    df_test[key] = pd.Categorical(df_object[key]).codes+1 # Adding 1 to avoid -1 for 'Unknown'

# Update the original dataframe with the encoded values 
df_test.update(df_test)
    

In [59]:
if df_test.isna().sum().any() > 0:
    print("There is still null values in the Dataset")
else:
    print("No null Values in the dataset")

No null Values in the dataset


In [76]:
prices = sk_model.predict(df_test)
prices

array([121198.4002132 , 157763.01587779, 180070.98563835, ...,
       173484.4119621 , 120710.64440836, 237957.47081074])

In [77]:
submission_sk = pd.DataFrame()
submission_sk["Id"] = df_test["Id"]
submission_sk["SalePrice"] = prices
submission_sk

,Id,SalePrice
0,1461,121198.400213
1,1462,157763.015878
2,1463,180070.985638
3,1464,177260.858875
4,1465,205544.495009
...,...,...
1454,2915,74186.437984
1455,2916,74494.412901
1456,2917,173484.411962
1457,2918,120710.644408


In [79]:
submission_sk.to_csv("submission_sk.csv" , index=False)